<a href="https://colab.research.google.com/github/entruv/code_stream/blob/master/Semantic_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install -qq wandb


     |████████████████████████████████| 1.4MB 4.8MB/s 
     |████████████████████████████████| 460kB 54.5MB/s 
     |████████████████████████████████| 102kB 15.5MB/s 
     |████████████████████████████████| 112kB 56.5MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 


## Objectif
We want to log machine ML predictions of a segmentation problem on the cloud, during training. For that we will need Callbacks and the wanndb library

## Resources
- https://github.com/ucbdrive/bdd-data/blob/master/bdd_data/label.py
- https://docs.fast.ai/
- https://github.com/borisdayma/semantic-segmentation


* [Semantic segmentation API →](https://docs.wandb.com/library/log#logging-image-masks-semantic-segmentation)
* [Read more about the problem and modeling approaches →](https://app.wandb.ai/stacey/deep-drive/reports/The-View-from-the-Driver's-Seat--Vmlldzo1MTg5NQ)
* [Repo: Semantic Segmentation for Self-Driving Cars→](https://github.com/borisdayma/semantic-segmentation)



- download data: https://storage.googleapis.com/wandb_datasets/BDD100K_seg_demo.zip


In [4]:
#login to the platform
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Downlaoding Berkeley Deep drive dataset
https://bdd-data.berkeley.edu/

In [5]:
!curl -SL -qq https://storage.googleapis.com/wandb_datasets/BDD100K_seg_demo.zip > BDD100K_seg_demo.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  958M  100  958M    0     0   132M      0  0:00:07  0:00:07 --:--:--  136M


In [0]:
#unzip this data
!unzip -qq BDD100K_seg_demo.zip

In [7]:
segmentation_classes = [
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car',
    'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'void'
]

def get_index_label_dictionary(labels):
  index_label_dictionary = {}
  for i, label in enumerate(labels):
    index_label_dictionary[i] = label
  return index_label_dictionary
get_index_label_dictionary(segmentation_classes)

{0: 'road',
 1: 'sidewalk',
 2: 'building',
 3: 'wall',
 4: 'fence',
 5: 'pole',
 6: 'traffic light',
 7: 'traffic sign',
 8: 'vegetation',
 9: 'terrain',
 10: 'sky',
 11: 'person',
 12: 'rider',
 13: 'car',
 14: 'truck',
 15: 'bus',
 16: 'train',
 17: 'motorcycle',
 18: 'bicycle',
 19: 'void'}

In [0]:
from pathlib import Path
path_data = Path("segment_demo")
path_labels = path_data / 'labels'
path_images = path_data / 'images'

In [9]:
filename_img = "2c2e25a9-cacc9ee2.jpg"
filename_label = "4804b6f7-00000000_train_id.png"

def get_labels_from_image(filename_img):
  filename_label = filename_img.split(".")[0] + "_train_id.png"
  return filename_label
print(get_labels_from_image(filename_img))


#x.parts[-2] is equal to val or train
get_y_fn = lambda x: path_labels / x.parts[-2] / f'{x.stem}_train_id.png'

2c2e25a9-cacc9ee2_train_id.png


In [0]:
sample_data_ratio = 0.01 #small ratio of our dataset for fast iteration
from fastai.vision import *

In [0]:
generator_images = SegmentationItemList.from_folder(path_images).use_partial_data(sample_data_ratio)
generator_images = generator_images.split_by_folder(train="train", valid="val").label_from_func(get_y_fn, classes=segmentation_classes)

## Creating a callback to log predictions during training time!


In [12]:
import wandb
labels = get_index_label_dictionary(segmentation_classes)
labels

{0: 'road',
 1: 'sidewalk',
 2: 'building',
 3: 'wall',
 4: 'fence',
 5: 'pole',
 6: 'traffic light',
 7: 'traffic sign',
 8: 'vegetation',
 9: 'terrain',
 10: 'sky',
 11: 'person',
 12: 'rider',
 13: 'car',
 14: 'truck',
 15: 'bus',
 16: 'train',
 17: 'motorcycle',
 18: 'bicycle',
 19: 'void'}

In [0]:
def mask_wb_format(true_img, pred_mask, true_mask):
  return wandb.Image(true_img, masks={
      "prediction": {"mask_data": pred_mask, "class_labels": labels},
      "ground_truth": {"mask_data": true_mask, "class_labels": labels}
  })

In [0]:
def torch_to_human(torch_image):
  return image2np(torch_image.data * 255).astype(np.uint8)

In [0]:
def unbatch(prediction):
  return prediction[0]

In [0]:
import numpy as np
class LogImagesCallback(Callback):
  def __init__(self, learn):
    self.learn = learn
    self.number_preds_to_log = 5
  
  def on_epoch_end(self, **kwargs):
    #get some examples from the validation set
    #make model prediction to this specific examples
    # logs this predictions for vizualizing them
    input_batch = self.learn.data.valid_ds[:self.number_preds_to_log]
    mask_list = [] #store the mask prediction 8
    for i, img_pair in enumerate(input_batch):
      image_torch_format = img_pair[0]
      image_human_format = torch_to_human(image_torch_format.data)

      mask_torch_format = img_pair[1]
      mask_human_format = torch_to_human(mask_torch_format)

      prediction_torch_format = unbatch(learn.predict(image_torch_format))
      prediction_human_format = torch_to_human(prediction_torch_format)

      mask_wb = mask_wb_format(image_human_format, prediction_human_format, mask_human_format)
      mask_list.append(mask_wb)
    wandb.log({"prediction": mask_list})

#calculons l'accuracy d'une prediction d'un mask entier
#pour faire simple. on va utiliser 1 accuracy pour chaque probleme specifique:
#1. overall accuracy
# 2. accuracy to specific classes: traffic light and traffic signs


def accuracy_mask(input, target):
  target = target.squeeze(1)
  mask = target != void_code
  try:
    i = (input.argmax(dim=1)[mask] == target[mask]).float()
    m_i = i.mean()
    return m_i
  except:
    return torch.tensor([0.0])

void_code = 19
def accuracy_traffic(input, target):
  #renvoie accuracy que en regardant les classes de pixels qui qui nous interessent
  target = target.squeeze(1)
  mask_pole = target == 5
  mask_light = target == 6
  mask_sign = taget = 7
  mask_traffic = mask_pole | mask_light | mask_sign
  try:
    i = (input.argmax(dim=1)[mask_traffic] == target[mask_traffic]).float()
    m_i = i.mean()
    return m_i
  except:
    return torch.tensor([0.0])




In [40]:
generator_images

LabelLists;

Train: LabelList (68 items)
x: SegmentationItemList
Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640)
y: SegmentationLabelList
ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640)
Path: segment_demo/images;

Valid: LabelList (12 items)
x: SegmentationItemList
Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640),Image (3, 360, 640)
y: SegmentationLabelList
ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640),ImageSegment (1, 360, 640)
Path: segment_demo/images;

Test: None

In [27]:
np.array((720, 1280)) / 2 #image resized for better performance

array([360., 640.])

In [28]:
project_name = "ai_education_cars"
wandb.init(project=project_name)



W&B Run: https://app.wandb.ai/aieducation/ai_education_cars/runs/6afcrj7g

In [0]:
config = wandb.config
config.framework = "fast.ai"
config.img_size = (360, 640)

#training config:

config.batch_size = 2
config.epochs = 10

config.encoder = "resnet18"
encoder = models.resnet18


#hyperparameter config
config.pretrained = True
config.one_cycle = True ##technique de fastaiAi pour entrainer un modele rapidement
config.learning_rate = 10e-3 #learning rate for training

#change our generator to be as the same format than the pretrained model configuration
#par example, en ayant la meme variance et moyenne des pixels des images original

data = (generator_images.transform(get_transforms(), size=config.img_size, tfm_y=True)
          .databunch(bs=config.batch_size).normalize(imagenet_stats))


## Quel sont les tailles de notre jeu de donnee?


In [0]:
assert len(data.train_ds), len(data.valid_ds) == (72, 8)
config.num_train =len(data.train_ds)
config.num_valid = len(data.valid_ds)

## Define the learner object containing our model and our data generator, and our metrtics, and all of our configurations

We want to use a UNET, since it's really simple to understand to train, and give us excellent results most of the time (this is a standard in the DL community

In [0]:
from wandb.fastai import WandbCallback

In [0]:
learn = unet_learner(data, 
                     encoder,
                     pretrained=config.pretrained,
                     metrics=[accuracy_mask, accuracy_traffic],
                     callback_fns=partial(WandbCallback, monitor="accuracy_mask"))


In [41]:
#lancer un entrainement pour tester
learn.fit_one_cycle(config.epochs, max_lr=config.learning_rate, callbacks=[LogImagesCallback(learn)])

epoch,train_loss,valid_loss,accuracy_mask,accuracy_traffic,time
0,1.628824,1.461188,0.605997,0.512994,00:08
1,2.365018,2.122516,0.395676,0.334718,00:08
2,66.685364,453.848389,0.156762,0.131877,00:08
3,60.129894,1.893581,0.512345,0.433534,00:08
4,30.234325,1.594792,0.566142,0.471654,00:08
5,15.879984,1.574470,0.603945,0.512098,00:08
6,8.800311,1.544633,0.575216,0.499188,00:08
7,5.243555,1.492679,0.523918,0.528167,00:08
8,3.462804,1.470247,0.558684,0.544435,00:08
9,2.565111,1.473715,0.553366,0.540015,00:08


Better model found at epoch 0 with accuracy_mask value: 0.6059972643852234.
Loaded best saved model from /content/wandb/run-20200418_150945-6afcrj7g/bestmodel.pth


In [50]:

learn.fit(50, callbacks=[LogImagesCallback(learn)])

epoch,train_loss,valid_loss,accuracy_mask,accuracy_traffic,time
0,1.007755,1.074074,0.679444,0.674291,00:08
1,1.048495,0.992763,0.715279,0.687247,00:08
2,1.027543,0.901577,0.795731,0.726512,00:08
3,1.020683,0.920360,0.789755,0.710803,00:08
4,1.020837,0.862899,0.781422,0.729225,00:08
5,1.037398,0.940768,0.786016,0.696242,00:08
6,1.031438,0.939218,0.790135,0.705595,00:08
7,1.034922,0.993306,0.759501,0.722480,00:08
8,1.003618,0.897275,0.777679,0.720437,00:08
9,0.994724,0.866949,0.776660,0.724916,00:08


Better model found at epoch 0 with accuracy_mask value: 0.6794437766075134.
Better model found at epoch 1 with accuracy_mask value: 0.7152793407440186.
Better model found at epoch 2 with accuracy_mask value: 0.7957313656806946.
Loaded best saved model from /content/wandb/run-20200418_150945-6afcrj7g/bestmodel.pth


KeyboardInterrupt: ignored

In [51]:
#entrainement devrai etre plus long car on doit entrainer tout les layers
learn.unfreeze()
learn.fit(50, callbacks=[LogImagesCallback(learn)])

epoch,train_loss,valid_loss,accuracy_mask,accuracy_traffic,time
0,1.976862,1.579448,0.625310,0.552142,00:09
1,1.626050,1.190209,0.720588,0.619038,00:08
2,1.392650,1.029826,0.761854,0.697900,00:08
3,1.270590,0.955022,0.791322,0.695514,00:08
4,1.173099,0.909525,0.789759,0.690378,00:08
5,1.096749,0.901028,0.790715,0.717274,00:08
6,1.020317,0.857296,0.790685,0.744716,00:08
7,0.970772,0.951918,0.788090,0.731178,00:08
8,0.947925,0.915233,0.790052,0.729793,00:08
9,0.951604,0.812246,0.796640,0.741618,00:08


Better model found at epoch 0 with accuracy_mask value: 0.6253097057342529.
Better model found at epoch 1 with accuracy_mask value: 0.7205879092216492.
Better model found at epoch 2 with accuracy_mask value: 0.7618544101715088.
Better model found at epoch 3 with accuracy_mask value: 0.7913224697113037.
Better model found at epoch 9 with accuracy_mask value: 0.7966402173042297.
Better model found at epoch 11 with accuracy_mask value: 0.800983190536499.
Better model found at epoch 13 with accuracy_mask value: 0.8204328417778015.
Better model found at epoch 14 with accuracy_mask value: 0.8216564655303955.
Better model found at epoch 17 with accuracy_mask value: 0.8253299593925476.
Better model found at epoch 24 with accuracy_mask value: 0.8334000706672668.
Better model found at epoch 25 with accuracy_mask value: 0.8438034653663635.
Better model found at epoch 32 with accuracy_mask value: 0.8503802418708801.
Loaded best saved model from /content/wandb/run-20200418_150945-6afcrj7g/bestmodel